# Add time series to `System`

**Originally Contributed by**: Clayton Barrows

## Introduction

An example of how to parse add time series data to a `System` using [PowerSystems.jl](github.com/NREL-SIIP/PowerSystems.jl)

For example, a `System` created by [parsing a MATPOWER file](https://nbviewer.jupyter.org/github/NREL-SIIP/SIIPExamples.jl/blob/master/notebook/2_PowerSystems_examples/parse_matpower.ipynb)
doesn't contain any time series data. So a user may want to add time series to the `System`
### Dependencies
Let's use the 5-bus dataset we parsed in the MATPOWER example

In [1]:
using SIIPExamples
using PowerSystems
using JSON3

pkgpath = dirname(dirname(pathof(SIIPExamples)))
include(joinpath(pkgpath, "test", "2_PowerSystems_examples", "02_parse_matpower.jl"))

[ Info: Correcting vm in bus 1 to 1.07762 to match generator set-point
[ Info: Correcting vm in bus 3 to 1.1 to match generator set-point
[ Info: Correcting vm in bus 4 to 1.06414 to match generator set-point
[ Info: Correcting vm in bus 10 to 1.06907 to match generator set-point
[ Info: Correcting vm in bus 3 to 1.0 to match generator set-point
┌ Error: Generator voltage set-points for bus 3 are inconsistent. This can lead to unexpected results
└ @ PowerSystems ~/.julia/packages/PowerSystems/gGFFl/src/parsers/pm_io/matpower.jl:245
[ Info: Correcting vm in bus 10 to 1.0 to match generator set-point
┌ Error: Generator voltage set-points for bus 10 are inconsistent. This can lead to unexpected results
└ @ PowerSystems ~/.julia/packages/PowerSystems/gGFFl/src/parsers/pm_io/matpower.jl:245
[ Info: extending matpower format with data: areas 1x3
[ Info: extending matpower format with data: gen_name 7x4
[ Info: extending matpower format by appending matrix "gen_name" in to "gen"
[ Info: rever

System
┌───────────────────┬─────────────┐
│ Property          │ Value       │
├───────────────────┼─────────────┤
│ System Units Base │ SYSTEM_BASE │
│ Base Power        │ 100.0       │
│ Base Frequency    │ 60.0        │
│ Num Components    │ 30          │
└───────────────────┴─────────────┘

Static Components
┌──────────────────────────┬───────┬────────────────────────┬───────────────┐
│ Type                     │ Count │ Has Static Time Series │ Has Forecasts │
├──────────────────────────┼───────┼────────────────────────┼───────────────┤
│ Arc                      │ 6     │ false                  │ false         │
│ Area                     │ 1     │ false                  │ false         │
│ Bus                      │ 5     │ false                  │ false         │
│ Line                     │ 5     │ false                  │ false         │
│ LoadZone                 │ 1     │ false                  │ false         │
│ PhaseShiftingTransformer │ 2     │ false                  │ false         │
│ PowerLoad                │ 3     │ false                  │ false         │
│ RenewableDispatch        │ 2     │ false                  │ false         │
│ ThermalStandard          │ 5     │ false                  │ false         │
└──────────────────────────┴───────┴────────────────────────┴───────────────┘

### Define pointers to time series files
For example, if we want to add a bunch of time series files, say one for each load and
one for each renewable generator, we need to define pointers to each .csv file containing
the time series in the following format (PowerSystems.jl also supports a CSV format for this file)

In [2]:
FORECASTS_DIR = joinpath(base_dir, "forecasts", "5bus_ts")
fname = joinpath(FORECASTS_DIR, "timeseries_pointers_da.json")
open(fname, "r") do f
    JSON3.@pretty JSON3.read(f)
end

[
  {
                           "simulation": "DAY_AHEAD",
                           "resolution": 3600,
                             "category": "Generator",
                       "component_name": "SolarBusC",
                               "module": "PowerSystems",
                                 "type": "SingleTimeSeries",
                                 "name": "max_active_power",
            "scaling_factor_multiplier": "get_max_active_power",
     "scaling_factor_multiplier_module": "PowerSystems",
                 "normalization_factor": 1,
                            "data_file": "./gen/Renewable/PV/da_solar5.csv"
  },
  {
                           "simulation": "DAY_AHEAD",
                           "resolution": 3600,
                             "category": "Generator",
                       "component_name": "WindBusA",
                               "module": "PowerSystems",
                                 "type": "SingleTimeSeries",
                             

### Read and assign time series to `System` using these parameters.

In [3]:
add_time_series!(sys, fname)
sys

System
┌───────────────────┬─────────────┐
│ Property          │ Value       │
├───────────────────┼─────────────┤
│ System Units Base │ SYSTEM_BASE │
│ Base Power        │ 100.0       │
│ Base Frequency    │ 60.0        │
│ Num Components    │ 30          │
└───────────────────┴─────────────┘

Static Components
┌──────────────────────────┬───────┬────────────────────────┬───────────────┐
│ Type                     │ Count │ Has Static Time Series │ Has Forecasts │
├──────────────────────────┼───────┼────────────────────────┼───────────────┤
│ Arc                      │ 6     │ false                  │ false         │
│ Area                     │ 1     │ false                  │ false         │
│ Bus                      │ 5     │ false                  │ false         │
│ Line                     │ 5     │ false                  │ false         │
│ LoadZone                 │ 1     │ false                  │ false         │
│ PhaseShiftingTransformer │ 2     │ false                  │ false         │
│ PowerLoad                │ 3     │ true                   │ false         │
│ RenewableDispatch        │ 2     │ true                   │ false         │
│ ThermalStandard          │ 5     │ false                  │ false         │
└──────────────────────────┴───────┴────────────────────────┴───────────────┘

Time Series Summary
┌──────────────────────────────────┬────────────┐
│ Property                         │ Value      │
├──────────────────────────────────┼────────────┤
│ Components with time series data │ 5          │
│ Total StaticTimeSeries           │ 5          │
│ Total Forecasts                  │ 0          │
│ Resolution                       │ 60 minutes │
└──────────────────────────────────┴────────────┘

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*